**Title: Star Classification**

**Introduction**

There are billions of stars in our sky and every star is unique. There are brown, white, and red dwarfs, main sequence stars like the sun and also hyper and super giants. Each of these star types has unique characteristics which can be used to classify their type. Astronomers typically use features such as the stars Luminosity, Radius, and Temperature to determine its type. In this data analysis, we will look at a star dataset which includes features of 240 stars which includes 40 stars from each of the 6 categories. The features of stars this dataset includes is the surface temperature in Kelvins, the luminosity in solar luminosity, radius in solar radius, visual magnitude (How bright the star appears from a distance of 10 Kpc), star type, star colour, and stellar class. Luminosity is the amount of energy generated per second by the star and is given in stellar units. This means that a luminosity of 100 means 100 times more luminous than the sun. The radius is also how big it is in stellar units, and a radius of 10 means 10 times bigger than the sun. Researchers have used the Hertzsprung-Russell Diagram (A plot of Luminosity vs. Temperature) to plot star features in order to determine a star’s type. To further our understanding of this diagram, we want to utilize k-nn classification to imitate aspects of the diagram and predict a new star’s type. The question we will address is whether we can build a model which effectively, consistently, and accurately classifies a new star’s type using k-nn classification to classify stars based on it's luminosity, radius, and temperature. 


**Methods & Results**

In this data analysis, the plan is to build a model using temperature, radius, and luminosity. To start off we will set our seed value (2022) and load in all the libraries we need for this data analysis

In [ ]:
#Load all libraries and set plot dimensions
set.seed(2022)
options(repr.plot.height = 8, repr.plot.width = 10)
library(tidyverse)
library(repr)
library(dplyr)
library(tidymodels)

**Load Dataset**

Now we need to load in our dataset and analyze it by looking at first 6 and last 6 observations using tail and head functions. 

In [ ]:
#Load data frame and changed the column names to more convenient
url <- "https://docs.google.com/spreadsheets/d/e/2PACX-1vR0bAqJh1jznOVyOZbHPL3kLbFjX6YzCWwMh3vMZEeBZq95tdDfcN9vqDP6XUf2j167vXr4oxnDvhb4/pub?gid=1536482276&single=true&output=csv"
star <- read_csv(url)

head(star)
tail(star)

**Clean and Wrangle Dataset**

Looking at the dataset, the column names have spaces so it would be a good idea to rename the columns to more convenient names, to make it more convenient to use with functions such as mutate and filter. We also want to change the type of the variable we are trying to classify which is star_type to a factor. The person who created the dataset decided to map the star type in star_type column as a number. They mapped Brown Dwarfs as 0, Red Dwarfs as 1, White Dwarfs as 2, Main Sequence stars as 3, Super giants as 4, and Hyper Giants as 5. To make our data analysis easier to understand we will map the star types back to their actual name. Lastly, judging from these two tables it seems like it would be a great idea to apply the log10 function to our predictors which are luminosity, radius, and temperature because the smallest values and largest values of these variables differ by a very significant amount. For example the lowest values for luminosity are a lot smaller than 1 while the largest values are greater than 100,000 so for the purpose of this data analysis since applying log10 function to each individual predictor will only decrease the amount each value varies by in respect to each other and won't impact relationship between values of other predictor, applying log10 will not impact our results for the data analysis and will make our values simpler to work with and visualize. So lets apply all these changes. 

In [ ]:
#Rename columns to more convenient name
star_data <- star %>% 
    rename(star_type = "Star type",
           star_color = "Star color",
           spectral_class = "Spectral Class",
           temperature = "Temperature (K)",
           radius = "Radius(R/Ro)",
           luminosity = "Luminosity(L/Lo)",
           absolute_magnitude = "Absolute magnitude(Mv)") %>%
    tibble()


#Renamed numerical values of star type to their actual name 
star_data$star_type[star_data$star_type == 0] <- "Brown Dwarf"
star_data$star_type[star_data$star_type == 1] <- "Red Dwarf"
star_data$star_type[star_data$star_type == 2] <- "White Dwarf"
star_data$star_type[star_data$star_type == 3] <- "Main Sequence"
star_data$star_type[star_data$star_type == 4] <- "Super Giant"
star_data$star_type[star_data$star_type == 5] <- "Hyper Giant"

#Changed star_type to a factor and also applied log10 function to all predictors
star_data <- star_data %>%
    mutate(star_type = as_factor(star_type),
          luminosity = log10(luminosity),
          radius = log10(radius),
          temperature = log10(temperature))

sample_n(star_data, 6)

Since we will be performing a knn classification, before doing any exploratory data analysis of our data we only want access to the training dataset and hide the test dataset. So lets split our dataset by using 60% of original dataset for the training set and 40% for the test set so we can have a fairly good balance between building an accurate model and have a large enough test dataset so we can get a precise accuracy for our model. 

In [ ]:
#Split the data into a training set and testing set 
star_split <- initial_split(star_data, prop = 0.6, strata = star_type)
star_train <- training(star_split)
star_test <- testing(star_split)

**Exploratory Data Analysis: Dataset Summary**

Now that the dataset has been split into a training and testing dataset, lets summarize our dataset by looking at the average temperature, average density, and average radius for each star type, as well as the number of stars included for each star type. 

In [ ]:
#Plot indicating the number of observations in each star type and averages of all predictor variables
star_distribution <- star_train %>%
    select(temperature, radius, luminosity, star_type) %>%
    group_by(star_type) %>%
    summarize(stars_in_class = n(), luminosity_avg = mean(luminosity), radius_avg = mean(radius), temperature_avg = mean(temperature))
star_distribution

**Table 1**

We can see from Table 1 that there are 24 stars from each star type included and brown dwarfs have the lowest luminosity value on average and Hyper giants have the largest luminosity value, White dwarfs have the smallest average radius and Hyper Giants have the largest average radius, and Brown Dwarfs have the lowest temperature average and White dwarfs have the highest temperature average. Judging from the low variability of temperature for the different star types, it may not be as good of a predictor for star type in comparison to Luminosity and Radius since their average values vary more between star types.  

**Exploratory Data Analysis: Visualizations**

Now we will create three different visualizations to visualize the relationships between all the predictors and since we suspect that temperature might not be the best predictor for star type, we will create a histogram which shows the distribution temperature for all the star types. This will help us gain a better understanding of dataset and guide us in building the best model. 

In [ ]:
#Visualization of Luminosity vs. Temperature
star_plot <- star_train %>%
    ggplot(aes(x = temperature, y = luminosity, color = star_type)) + 
    geom_point() +
    ggtitle("Luminosity(Log Scaled) vs. Surface Temperature(Log Scaled)") +
    labs(x = "Surface Temperature(Log Scaled)", y = "Luminosity(Log Scaled)", color = "Star Type") +
    theme(text = element_text(size = 20))

star_plot 

Luminosity and temperature seems to have a multiple relationships going on. For Main Sequence Stars, Red dwarfs, and Brown dwarfs, there is a strong positive linear relationship and there seems to be a moderate positive relationship for White Dwarfs and Hyper Giants and Super Giants don't have a relationship at all. 

**Figure 1**

In [ ]:
#Visualization of Luminosity vs. Radius
star_plot_2 <- star_train %>%
    ggplot(aes(x = radius, y = luminosity, color = star_type)) + 
    geom_point() +
    ggtitle("Luminosity(Log Scaled) vs. Radius(Log Scaled)") +
    labs(x = "Radius(Log Scaled)", y = "Luminosity(Log Scaled)", color = "Star Type") +
    theme(text = element_text(size = 20))

star_plot_2

Luminosity and Radius have a strong positive non-linear relationship

**Figure 2**

In [ ]:
#Visualization of Temperature vs. Radius
star_plot_3 <- star_train %>%
    ggplot(aes(x = radius, y = temperature, color = star_type)) + 
    geom_point() +
    ggtitle("Surface Temperature(Log Scaled) vs. Radius(Log Scaled)") +
    labs(x = "Radius(Log Scaled)", y = "Surface Temperature(Log Scaled)", color = "Star Type") +
    theme(text = element_text(size = 20))

star_plot_3

**Figure 3**

Based off of figure 3, there is a Strong positive non-linear relationship between Temperature and Radius for Brown Dwarfs, Red Dwarfs, and Main Sequence stars but no relationship for the rest of the star types.

In [ ]:
#Plot to see distribution of star type and temperature
star_temp_plot <- star_train %>%
    ggplot(aes(x = temperature, fill = star_type)) +
    geom_histogram(position = "fill", color = "lightgrey", bins = 15) +
    ggtitle("Star Type Distribution vs. Temperature") +
    labs(x = "Temperature(Log Scaled)", y = "Star Type distribution", color = "Star Type") +
    theme(text = element_text(size = 20))

star_temp_plot

**Figure 4**

Based off of Figure 4, there is too much star type variability for temperature. Temperatures from 3.4 to 3.6 and from around 3.8 to 5 include up to 2-4 different star types.

**Conclusion from Exploratory Data Analysis**

After conducting the exploratory data analysis, based off of Table 1, Figure 1, Figure 3 and 4, we can hypothesize that Temperature may not be the best predictor as it might cause a some observations to be classified incorrectly. However, luminosity and radius seem to be very good predictors. So to start off by building a model by including temperature and then well build a model by excluding temperature and then we will apply cross validation to compare accuracies. 

**Performing Data Analysis**

We will create two models in for our data analysis and we will pick the model with the best accuracy when applying cross validation as the model which we will use to predict the star type of stars in the test set. We will use a 10 fold cross validation throughout our data analysis since it will be a good amount of folds to determine good accuracies for various K values and also won't be too slow. We will test K values from 1 to 20 in our data analysis. 

In [ ]:
#Star Vfold to apply cross-validation
star_vfold <- vfold_cv(star_train, v = 10, strata = star_type)
k_vals <- tibble(neighbors = seq(1,20))

Now lets create the model which includes Temperature. We will build a model specification, create a recipe where we predict stars using the star_train dataset and luminosity, radius, and temperature to predict the star type. We will put everything together in a workflow and use cross validation to find the best K value.

In [ ]:
#K-nearest neighbor model specification: pre cross-validation
knn_spec_temp <- nearest_neighbor(weight_func = "rectangular", neighbors = tune()) %>%
    set_engine("kknn") %>%
    set_mode("classification") 

#Recipe for star data and standardization
star_recipe_temp <- recipe(star_type ~ radius + luminosity + temperature, data = star_train) %>%
    step_scale(all_predictors()) %>%
    step_center(all_predictors())

#Put together everything into a workflow
star_fit_temp <- workflow() %>%
    add_recipe(star_recipe_temp) %>%
    add_model(knn_spec_temp) %>%
    tune_grid(resamples = star_vfold, grid = k_vals)

# #Collect the summary of our cross-validation
star_summary_temp <- star_fit_temp %>%
    collect_metrics() %>%
    filter(.metric == "accuracy")

In [ ]:
#Plot of accuracy vs. neighbors  
star_accuracy_plot <- star_summary_temp %>%
    ggplot(aes(x = neighbors, y = mean)) + 
    geom_point() +
    geom_line() +
    ggtitle("Accuracy vs. Neighbors") +
    labs(x = "Neighbors", y = "Accuracy") +
    theme(text = element_text(size = 20)) +
    scale_x_continuous(breaks = 1:20)

star_accuracy_plot

**Figure 5**

After visualizing accuracies for k values between 1 to 20, we can see that the optimal k value is 3. So lets extract that k value. 

In [ ]:
#Extracts best K value
k_min <- star_summary_temp %>%
    arrange(desc(mean)) %>% 
    slice(1) %>%
    select(neighbors) %>%
    pull()

k_min

Now lets retrain our model by making a new model specification with the best k value and apply cross validation to determine accuracy

In [ ]:
#K-nearest neighbor model specification: pre cross-validation
knn_spec_temp <- nearest_neighbor(weight_func = "rectangular", neighbors = k_min) %>%
    set_engine("kknn") %>%
    set_mode("classification") 

#Put together everything into a workflow
star_fit_temp <- workflow() %>%
    add_recipe(star_recipe_temp) %>%
    add_model(knn_spec_temp) %>%
    fit_resamples(resamples = star_vfold)

# #Collect the summary of our cross-validation
star_summary_temp <- star_fit_temp %>%
    collect_metrics() %>%
    filter(.metric == "accuracy")

star_summary_temp

**Table 2**

Table 2 shows us that our model is very good at predicting star type but to test our hypothesis we want to see if we can create a better model by excluding temperature as a predictor. Lets create our new model by specifying a new model specification, a new recipe where we use radius and luminosity to predict the star type and do cross validation to get the best k value.

In [ ]:
#K-nearest neighbor model specification: pre cross-validation
knn_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = tune()) %>%
    set_engine("kknn") %>%
    set_mode("classification") 

#Recipe for star data and standardization
star_recipe <- recipe(star_type ~ radius + luminosity, data = star_train) %>%
    step_scale(all_predictors()) %>%
    step_center(all_predictors())

#Put together everything into a workflow
star_fit <- workflow() %>%
    add_recipe(star_recipe) %>%
    add_model(knn_spec) %>%
    tune_grid(resamples = star_vfold, grid = k_vals)

# #Collect the summary of our cross-validation
star_summary <- star_fit %>%
    collect_metrics() %>%
    filter(.metric == "accuracy")

In [ ]:
#Plot of accuracy vs. neighbors  
star_accuracy_plot <- star_summary %>%
    ggplot(aes(x = neighbors, y = mean)) + 
    geom_point() +
    geom_line() +
    ggtitle("Accuracy vs. Neighbors") +
    labs(x = "Neighbors", y = "Accuracy") +
    theme(text = element_text(size = 20)) +
    scale_x_continuous(breaks = 1:20)

star_accuracy_plot

**Figure 6**

Plot of accuracy vs. neighbors for this model shows us that k = 11 is the best k value so lets extract it. 

In [ ]:
#Best K value
k_min <- star_summary %>%
    arrange(desc(mean)) %>% 
    slice(1) %>%
    select(neighbors) %>%
    pull()

k_min

Now lets retrain our model by using our optimal k value and use cross validation to determine what our accuracy is for this new model. 

In [ ]:
#Retrained model with new k value 
knn_best_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = k_min) %>%
    set_engine("kknn") %>%
    set_mode("classification") 

#Fit our model using a workflow and obtain accuracy using cross-validation
star_best_accuracy <- workflow() %>%
    add_recipe(star_recipe) %>%
    add_model(knn_best_spec) %>%
    fit_resamples(resamples = star_vfold) %>%
    collect_metrics() %>%
    filter(.metric == "accuracy")

star_best_accuracy

**Table 3**

From Table 2 and 3, we can clearly see that our model which excludes temperature has a higher accuracy even though it's only around 1%. Since our new model is slightly better we will use that on our test dataset.

Lets fit our model onto the training data and use it to predict the star type for stars in our test data set and see what the accuracy is

In [ ]:
#Fit the model onto training set
star_best_fit <- workflow() %>%
    add_recipe(star_recipe) %>%
    add_model(knn_best_spec) %>%
    fit(data = star_train) 

#Predict star type for the test data set using our model
star_predictions <- star_best_fit %>%
    predict(star_test) %>%
    bind_cols(star_test)

In [ ]:
#Obtain the accuracy for the model
star_metrics <- star_predictions %>%
    metrics(truth = star_type, estimate = .pred_class) %>%
    filter(.metric == "accuracy")

star_metrics

**Table 4**

Table 4 shows us our model is around 94.8% accurate. Now lets use a confusion matrix to see where our model made mistakes. 

In [ ]:
#Create a confusion matrix for our predictions
star_conf_mat <- star_predictions %>%
    conf_mat(truth = star_type, estimate = .pred_class)

star_conf_mat

**Table 5**

Our model predicted Brown Dwarfs, White Dwarfs, Super Giants and Hyper Giants flawlessly but misclassified a few Red Dwarfs and one Main Sequence star but overall our model is looking very good. Lets create a visualization to show how our model classifies star based on regions of a plot for luminosity vs. radius. 

In [ ]:
# create the grid of Luminosity/Radius vals, and arrange in a data frame
lum_grid <- seq(min(star_data$luminosity), 
                max(star_data$luminosity), 
                length.out = 100)
rad_grid <- seq(min(star_data$radius), 
                max(star_data$radius), 
                length.out = 100)
asgrid <- tibble(expand.grid(luminosity = lum_grid, 
                                radius = rad_grid))

#Make predictions for the observations in grid
knnPredGrid <- predict(star_best_fit, asgrid)

# bind the predictions as a new column with the grid points
prediction_table <- bind_cols(knnPredGrid, asgrid) %>%
  rename(Class = .pred_class)

#A plot of luminosity vs. Temperature using our original dataset where each grid is classified as a star. 
wkflw_plot <-
  ggplot() +
  geom_point(data = star_data, 
             mapping = aes(x = radius, 
                           y = luminosity, 
                          color = star_type), 
             alpha = 0.75) +
  geom_point(data = prediction_table, 
             mapping = aes(x = radius, 
                           y = luminosity, 
                           color = Class), 
             alpha = 0.06, 
             size = 5) +
  labs(color = "Diagnosis", 
       x = "Radius (Log Scaled)", 
       y = "Luminosity (Log Scaled)") +
  scale_color_manual(labels = c("White Dwarf", "Main Sequence", "Brown Dwarf", "Red Dwarf", "Hyper Giant", "Super Giant"), 
                     values = c("azure4", "chartreuse4", "brown", "brown1", "darkorchid2", "deeppink1")) +
theme(text = element_text(size = 20)) + 
    ggtitle("Luminosity vs. Temperature")

wkflw_plot

**Figure 7**

Now lets create a bar graph where we visualize the proportionality of correct vs. incorrectely predicted stars for each star type. 

In [ ]:
#Plot showing distribution of each star types incorrect vs correctly predicted stars
star_accuracy <- star_best_fit %>%
    predict(star_test) %>%
    bind_cols(star_test) %>%
    mutate(prediction = ifelse(.pred_class == star_type, "correct", "incorrect")) %>%
    group_by(star_type, prediction) %>%
    summarize(n = n()) %>%
    ggplot(aes(x = star_type, y = n, fill = prediction)) +
    geom_bar(stat = "identity", position = "fill") +
    scale_fill_brewer(palette = "RdYlBu") +
    ggtitle("Predictions vs. Star Type") +
    labs(x = "Star Type", y = "Correct vs. Incorrect Proportion", color = "Predictions") +
    theme(text = element_text(size = 15))

star_accuracy

**Figure 8**